## Replication for "Asymmetry by Design", August 7, 2021

By Adam Goldstein and Charlie Eaton

## Table 3: Pooled OLS and MLE Estimates for Predatory Practices

Install STATA packages

In [1]:
ssc install estout

checking estout consistency and verifying not already installed...
installing into /Users/Charlie/Library/Application Support/Stata/ado/plus/...
installation complete.


Download datasets

In [2]:
copy "https://github.com/HigherEdData/asymmetry/raw/master/data/d_unitidasymmetry.dta" ///
    data/d_unitidasymmetry.dta, replace

copy "https://github.com/HigherEdData/asymmetry/raw/master/data/d_opeidasymmetry.dta" ///
    data/d_opeidasymmetry.dta, replace

Report number of total observations and observations that are multi-brand with data for all control variables

In [3]:
quietly use data/d_unitidasymmetry, clear
qui gen missing=.
foreach var in tuitionall_c_w multi_brand system_under all_under online selective iclevel ///
pell_grants_per_fte_c_w white_share_w black_share_w hisp_share_w dpcttype_health ///
dpcttype_law dpcttype_culinarycosmetic dpcttype_arts dpcttype_tech dpcttype_biz ///
grad_rate_150_p_w year state_n {
   qui replace missing=1 if `var'==.
}
tab multi_brand if missing!=1 & year<2016






multi-brand |      Freq.     Percent        Cum.
------------+-----------------------------------
          0 |     12,375       73.20       73.20
          1 |      4,530       26.80      100.00
------------+-----------------------------------
      Total |     16,905      100.00


**********
Count 743 colleges operated under 49 multi-brand firms by the first year of operation by the multi-brand firm

In [4]:
bysort unitid: egen minmulti=min(year) if missing!=1 & multi_brand==1
tab year if year==minmulti


(73,672 missing values generated)


       year |      Freq.     Percent        Cum.
------------+-----------------------------------
       1995 |         29        3.90        3.90
       1996 |          1        0.13        4.04
       1997 |         12        1.62        5.65
       1998 |         58        7.81       13.46
       1999 |         23        3.10       16.55
       2000 |         25        3.36       19.92
       2001 |         34        4.58       24.50
       2002 |         40        5.38       29.88
       2003 |         40        5.38       35.26
       2004 |         63        8.48       43.74
       2005 |         30        4.04       47.78
       2006 |         40        5.38       53.16
       2007 |         61        8.21       61.37
       2008 |         56        7.54       68.91
       2009 |         25        3.36       72.27
       2010 |         75       10.09       82.37
       2011 |         72        9.69       92.06
       2012 |         42        

**********
Count 49 multi-brand firms by first year with multi-brand structure for firms with data for all controls

In [5]:
keep if missing!=1
collapse (first) multi_brand systemnm, by(systemid year)
bysort systemid: egen minmulti=min(year) if multi_brand==1
tab year if year==minmulti


(61,297 observations deleted)


(7,181 missing values generated)


       year |      Freq.     Percent        Cum.
------------+-----------------------------------
       1995 |          9       18.37       18.37
       1997 |          2        4.08       22.45
       1998 |          4        8.16       30.61
       2000 |          5       10.20       40.82
       2001 |          5       10.20       51.02
       2002 |          3        6.12       57.14
       2003 |          4        8.16       65.31
       2004 |          3        6.12       71.43
       2006 |          3        6.12       77.55
       2008 |          3        6.12       83.67
       2010 |          3        6.12       89.80
       2011 |          2        4.08       93.88
       2012 |          2        4.08       97.96
       2013 |          1        2.04      100.00
------------+-----------------------------------
      Total |         49      100.00


In [6]:
set more off

est clear 

quietly use data/d_unitidasymmetry, clear

quietly collapse (rawsum) sftesale sftetotl (mean) all_under system_under grad_rate_150_p_w ///
loan_amount_borrower_c_w tuitionall_c_w selective white_share_w black_share_w hisp_share_w ///
pell_grants_per_fte_c_w ft_faculty_per_100fte_w dpcttype_health dpcttype_law ///
dpcttype_culinarycosmetic dpcttype_arts dpcttype_tech dpcttype_biz online ///
(first) state_n (min) iclevel (max) law_enf_frst_this_yr multi_brand, by(systemid year)

quietly gen fracsales= sftesale /sftetotl * 100

quietly label var fracsales "% employees sales"

quietly xtset systemid year

quietly eststo: reg fracsales multi_brand system_under online selective ib3.iclevel ///
pell_grants_per_fte_c_w white_share_w black_share_w hisp_share_w dpcttype* i.year i.state_n, cluster(systemid)
qui estadd scalar schools = e(N_clust)

In [7]:
quietly use data/d_unitidasymmetry, clear
quietly xtset unitid year

qui gen missing=.
foreach var in tuitionall_c_w multi_brand system_under all_under online selective iclevel ///
pell_grants_per_fte_c_w white_share_w black_share_w hisp_share_w dpcttype_health ///
dpcttype_law dpcttype_culinarycosmetic dpcttype_arts dpcttype_tech dpcttype_biz ///
year state_n grad_rate_150_p_w {
 quietly replace missing=1 if `var'==.
}

foreach var in tuitionall_c_w loan_amount_borrower_c_w {
qui egen count = group(unitid) if missing!=1 & `var'!=.
qui sum  count
qui local l`var'=r(max)
qui drop count
qui eststo: reg `var' multi_brand system_under all_under online selective ib3.iclevel ///
pell_grants_per_fte_c_w white_share_w black_share_w hisp_share_w dpcttype* grad_rate_150_p_w i.year i.state_n, cluster(systemid)
qui estadd scalar schools = `l`var''
}

quietly use data/d_unitidasymmetry, clear

quietly xtset unitid year
quietly gen l2multi_brand=l2.multi_brand
quietly replace multi_brand=l2multi_brand

qui  gen missing=.
foreach var in tuitionall_c_w multi_brand system_under all_under online selective iclevel ///
pell_grants_per_fte_c_w white_share_w black_share_w hisp_share_w dpcttype_health ///
dpcttype_law dpcttype_culinarycosmetic dpcttype_arts dpcttype_tech dpcttype_biz year state_n {
 quietly replace missing=1 if `var'==.
}

qui egen count = group(unitid) if missing!=1 & ft_faculty_per_100fte_w!=.
qui  sum  count
qui   local lfac=r(max)

quietly eststo: reg ft_faculty_per_100fte_w multi_brand system_under all_under online selective ib3.iclevel ///
pell_grants_per_fte_c_w white_share_w black_share_w hisp_share_w dpcttype* tuitionall_c_w ib1.year i.state_n, cluster(systemid)
qui estadd scalar schools =`lfac'

qui gen fracsales=.
quietly label var fracsales "% employees sales"

quietly esttab using tables/t3a_asymmetryols20200912.rtf, title(Table 3A: OLS estimates for predatory practices) ///
 keep(multi_brand system_under all_under online selective 1.iclevel 2.iclevel ///
pell_grants_per_fte_c_w white_share_w black_share_w hisp_share_w dpcttype_health dpcttype_law ///
dpcttype_culinarycosmetic dpcttype_arts dpcttype_tech dpcttype_biz grad_rate_150_p_w tuitionall_c_w _cons) ///
order(multi_brand multi_brand system_under all_under) ///
stats(r2 schools N_clust N, fmt(2 %9.0fc %9.0fc) label("r2" "schools/firms" N)) ///
se(%9.3fc) b(%9.3fc) nogaps  ///
 star(* 0.05 ** 0.01 *** 0.001) label varwidth(10) replace

In [8]:
%html
esttab, ///
 keep(multi_brand system_under all_under online selective 1.iclevel 2.iclevel ///
pell_grants_per_fte_c_w white_share_w black_share_w hisp_share_w dpcttype_health dpcttype_law ///
dpcttype_culinarycosmetic dpcttype_arts dpcttype_tech dpcttype_biz grad_rate_150_p_w tuitionall_c_w _cons) ///
order(multi_brand system_under all_under) se(%9.3fc) b(%9.3fc) nogaps ///
stats(r2 schools N, fmt(2 %9.0fc %9.0fc) label("r2" "schools/firms" N)) ///
star(* 0.05 ** 0.01 *** 0.001) html label varwidth(10)

This front-end or document format cannot display HTML

In [9]:
quietly use data/d_unitidasymmetry, clear
quietly est clear
quietly xtset unitid year
  set more off  

qui gen missing=.
foreach var in tuitionall_c_w multi_brand system_under all_under online selective iclevel ///
pell_grants_per_fte_c_w white_share_w black_share_w hisp_share_w dpcttype_health ///
dpcttype_law dpcttype_culinarycosmetic dpcttype_arts dpcttype_tech dpcttype_biz ///
year state_n {
 quietly replace missing=1 if `var'==.
}
    
foreach var in grad_rate_150_p4yr_w grad_rate_150_p2yr_w  {

qui egen count = group(unitid) if missing!=1 & `var'!=.
qui sum  count
qui local l`var'=r(max)
qui drop count

qui eststo: reg `var' multi_brand system_under all_under online selective ib3.iclevel ///
pell_grants_per_fte_c_w white_share_w black_share_w hisp_share_w dpcttype* tuitionall_c_w i.year ///
    i.state_n, cluster(systemid)
qui estadd scalar schools = `l`var''
}

In [10]:
quietly {
    quietly use data/d_opeidasymmetry, clear
keep if rankopeid==1
xtset opeid year

quietly gen l2multi_brand=l2.multi_brand
quietly replace multi_brand=l2multi_brand

qui  gen missing=.
foreach var in tuitionall_c_w multi_brand system_under all_under online selective iclevel ///
white_share_w black_share_w hisp_share_w dpcttype_health ///
dpcttype_law dpcttype_culinarycosmetic dpcttype_arts dpcttype_tech dpcttype_biz year ///
state_n {
 quietly replace missing=1 if `var'==.
}

qui egen count = group(opeid) if missing!=1 & mn_earn_wne_p6_c_w!=. & earn_pct_lo!=.
qui sum  count
qui local learn=r(max)
qui drop count
 
quietly eststo: reg mn_earn_wne_p6_c_w multi_brand   ///
all_under system_under online selective ib3.iclevel earn_pct_lo ///
white_share_w black_share_w hisp_share_w dpcttype* tuitionall_c_w i.year i.state_n ///
, cluster(systemid)
qui estadd scalar schools = `learn'
    
quietly replace earn_pct_lo=rpy_3yr_pct_lo

qui egen count = group(opeid) if missing!=1 & rpy_3yr_rt_supp_w!=. & earn_pct_lo!=.
qui sum  count
qui local lrepay=r(max)
qui drop count

qui eststo: reg rpy_3yr_rt_supp_w multi_brand ///
all_under system_under online selective ib3.iclevel earn_pct_lo ///
white_share_w black_share_w hisp_share_w dpcttype* tuitionall_c_w i.year i.state_n ///
, cluster(systemid)
qui estadd scalar schools = `lrepay'
    
}

In [11]:
quietly use data/d_opeidasymmetry, clear

qui replace pell_grants_per_fte_c_w=l1.pell_grants_per_fte_c_w if year==2016

quietly gen l2multi_brand=l2.multi_brand
quietly replace multi_brand=l2multi_brand

qui eststo: nbreg claimcount multi_brand ///
all_under system_under online selective ib3.iclevel ///
white_share_w black_share_w hisp_share_w dpcttype* ///
pell_grants_per_fte_c_w i.state_n, vce(cluster systemid) noomitted
qui estadd scalar schools = e(N)

In [12]:
label var  multi_brand "multi-brand"
label var system_under "firm enrollment (1,000s)"
label var all_under "campus enrollment (1,000s)"
label var online "online"
label var selective "selective admissions"
label define iclevel 1 "BA offered" 2 "AA offered" 
    label values iclevel iclevel
label var black_share_w "% Black"
label var hisp_share_w "% Hispanic"
label var white_share_w "% White"
label var dpcttype_arts "% degr. art"
label var dpcttype_biz "% degr. biz"
label var dpcttype_culinarycosmetic "% degr. personal services"
label var dpcttype_health "% degr. health"
label var dpcttype_law "% degr. law (undergrad)"
label var dpcttype_tech "% degr. tech"
label var tuitionall_c_w "tuition (1,000s)"
label var grad_rate_150_p_w "undergrad grad rate"
label var earn_pct_lo "% low-income"

In [13]:
%html
esttab, ///
 keep(multi_brand system_under all_under online selective 1.iclevel 2.iclevel ///
pell_grants_per_fte_c_w white_share_w black_share_w hisp_share_w dpcttype_health dpcttype_law ///
dpcttype_culinarycosmetic dpcttype_arts dpcttype_tech dpcttype_biz earn_pct_lo tuitionall_c_w _cons) ///
order(multi_brand system_under all_under) ///
mtitle("BA grad rate" "other grad rate" "earning (1,000s)" "% repay" "fraud claims" "legal action") ///
eqlabels(none) collabels(none) ///
stats(r2 r2_p schools N, fmt(2 2 %9.0fc %9.0fc) label("r2 / pseudo r2" "pseudo r2" "schools/firms" N)) ///
se(%9.3fc) b(%9.3fc) nogaps ///
 star(* 0.05 ** 0.01 *** 0.001) html label varwidth(10)

This front-end or document format cannot display HTML

In [14]:
esttab using tables/t3b_asymmetryols20200912.rtf, replace ///
 keep(multi_brand system_under all_under online selective 1.iclevel 2.iclevel ///
pell_grants_per_fte_c_w white_share_w black_share_w hisp_share_w dpcttype_health dpcttype_law ///
dpcttype_culinarycosmetic dpcttype_arts dpcttype_tech dpcttype_biz earn_pct_lo tuitionall_c_w _cons) ///
order(multi_brand system_under all_under) ///
mtitle("BA grad rate" "other grad rate" "earning (1,000s)" "% repay" "fraud claims" "legal action") ///
eqlabels(none) collabels(none) ///
stats(r2 r2_p schools N, fmt(2 2 %9.0fc %9.0fc) label("r2" "pseudo r2" "schools/firms" N)) ///
se(%9.3fc) b(%9.3fc) nogaps ///
 star(* 0.05 ** 0.01 *** 0.001) label varwidth(10)

(output written to tables/t3b_asymmetryols20200912.rtf)


In [15]:
quietly {
    est clear
    quietly use data/d_unitidasymmetry, clear  
quietly collapse (rawsum) sftesale sftetotl law_enf_frst_this_yr (mean) grad_rate_150_p_w ///
loan_amount_borrower_c_w tuitionall_c_w selective ///
white_share_w black_share_w hisp_share_w pell_grants_per_fte_c_w ft_faculty_per_100fte_w ///
dpcttype_health dpcttype_law dpcttype_culinarycosmetic dpcttype_arts dpcttype_tech ///
dpcttype_biz all_under online (first) state_n system_under (min) iclevel (max) multi_brand  ///
grad_rate_150_p4yr_w grad_rate_150_p2yr_w, by(systemid year)

replace law_enf_frst_this_yr=1 if law_enf_frst_this_yr<. & law_enf_frst_this_yr>0

xtset systemid year

quietly xtset systemid year

eststo: logistic f2.law_enf_frst_this_yr multi_brand system_under online selective ib3.iclevel ///
pell_grants_per_fte_c_w white_share_w black_share_w hisp_share_w dpcttype*, cluster(systemid)
qui estadd scalar schools = e(N_clust1)
    
eststo: logistic f2.law_enf_frst_this_yr multi_brand system_under online selective ib3.iclevel ///
pell_grants_per_fte_c_w white_share_w black_share_w hisp_share_w dpcttype* i.year, cluster(systemid)
qui estadd scalar schools = e(N_clust1)
    
eststo: logistic f2.law_enf_frst_this_yr multi_brand system_under online selective ib3.iclevel ///
pell_grants_per_fte_c_w white_share_w black_share_w hisp_share_w dpcttype* i.year i.state_n, cluster(systemid)
qui estadd scalar schools = e(N_clust1)
    
quietly label var pell_grants_per_fte_c_w "Pell $ / student"

label var system_under "firm enrollment (1,000s)"
label var all_under "campus enrollment (1,000s)"
label var online "online"
label var selective "selective admissions"
    label drop iclevel
label define iclevel 1 "BA offered" 2 "AA offered" 
    label values iclevel iclevel
label var black_share_w "% Black"
label var hisp_share_w "% Hispanic"
label var white_share_w "% White"
label var dpcttype_arts "% degr. art"
label var dpcttype_biz "% degr. biz"
label var dpcttype_culinarycosmetic "% degr. personal services"
label var dpcttype_health "% degr. health"
label var dpcttype_law "% degr. law (undergrad)"
label var dpcttype_tech "% degr. tech"
label var tuitionall_c_w tuition
label var loan_amount_borrower_c_w borrowing
label var ft_faculty_per_100fte_w "# faculty"
label var law_enf_frst_this_yr "law enforcement"
label var grad_rate_150_p4yr_w "BA grad rate"
label var grad_rate_150_p2yr_w "other grad rate"
label var tuitionall_c_w "tuition (1,000s)"
label var grad_rate_150_p_w "undergrad grad rate"

esttab using tables/t3b_legaloddratios20200912.rtf, eform replace ///
 keep(multi_brand system_under all_under online selective 1.iclevel ///
2.iclevel pell_grants_per_fte_c_w white_share_w black_share_w ///
hisp_share_w dpcttype_health dpcttype_law dpcttype_culinarycosmetic ///
dpcttype_arts dpcttype_tech dpcttype_biz _cons) order(multi_brand ///
system_under all_under) ///
mtitle("odds ratio" "year fixed effects" "year and state fixed effects") ///
eqlabels(none) collabels(none) ///
stats(r2 r2_p N_clust N, fmt(2 2 %9.0fc %9.0fc) label("r2" "pseudo r2" "schools/firms" N)) ///
se(%9.3fc) b(%9.3fc) nogaps ///
 star(* 0.05 ** 0.01 *** 0.001) label varwidth(10)
}

In [16]:
%html
esttab, eform html ///
keep(multi_brand system_under online selective 1.iclevel 2.iclevel ///
pell_grants_per_fte_c_w white_share_w black_share_w hisp_share_w dpcttype_health dpcttype_law ///
dpcttype_culinarycosmetic dpcttype_arts dpcttype_tech dpcttype_biz _cons) ///
order(multi_brand system_under all_under) ///
mtitle("odds ratio" "year fixed effects" "year and state fixed effects") ///
eqlabels(none) collabels(none) ///
stats(r2_p N_clust N, fmt(2 %9.0fc %9.0fc) label("pseudo r2" "schools/firms" N)) ///
se(%9.3fc) b(%9.3fc) nogaps ///
 star(* 0.05 ** 0.01 *** 0.001) label varwidth(10)

This front-end or document format cannot display HTML